In [177]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [178]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"
# endpoint = 

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
# query_url=f"{base_url}{CME}?start_date=["startDate"]&end_date=["endDate"]&api_key=["NASA_API_KEY"]

cme_url=f"{base_url}{CME}?start_date={startDate}]&end_date={endDate}&api_key={NASA_API_KEY}"

In [179]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_url)


In [180]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()


In [181]:
# Preview the first result in JSON format
print(json.dumps(cme_json[:1], indent=4))



# Use json.dumps with argument indent=4 to format data


[
    {
        "activityID": "2024-09-23T02:00:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2024-09-23T02:00Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "CME visible to the NW in SOHO LASCO C2/C3. Not clearly visible in STEREO A COR2 imagery, and possibly obscured by the bright streamer. The potential source is opening field lines best seen in GOES SUVI 284 at 2024-09-23T02:00Z, on the far side.",
        "submissionTime": "2024-09-23T18:44Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/33551/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2024-09-23T08:47Z",
                "latitude": 36.0,
                "longitude": null,


In [182]:
# Convert cme_json to a Pandas DataFrame 
df = pd.DataFrame(cme_json)


# Keep only the columns: activityID, startTime, linkedEvents

cols = ["activityID", "startTime", "linkedEvents"]

df= df[cols]

df.head()


,activityID,startTime,linkedEvents
0,2024-09-23T02:00:00-CME-001,2024-09-23T02:00Z,None
1,2024-09-23T09:12:00-CME-001,2024-09-23T09:12Z,None
2,2024-09-23T20:48:00-CME-001,2024-09-23T20:48Z,[{'activityID': '2024-09-28T07:00:00-IPS-001'}]
3,2024-09-24T04:12:00-CME-001,2024-09-24T04:12Z,None
4,2024-09-25T04:53:00-CME-001,2024-09-25T04:53Z,None


In [183]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs

df_clean= df.dropna(subset=["linkedEvents"])


In [171]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
linkedEvents_list = []

# Iterate over each index in the DataFrame

for index in df_clean.index:
    activityID = df_clean.loc[index, "activityID"]
    startTime = df_clean.loc[index, "startTime"]
    linked_events = df_clean.loc[index, "linkedEvents"]

    for event in linked_events:
            dict = {
                "activityID": activityID,
                "startTime": startTime,
                "linkedEvents": event
            }
            linkedEvents_list.append(dict)

    # Iterate over each dictionary in the list
    
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
      
# Create a new DataFrame from the expanded rows

df_new = pd.DataFrame(linkedEvents_list)
df_new.head()


,activityID,startTime,linkedEvents
0,2024-09-23T20:48:00-CME-001,2024-09-23T20:48Z,{'activityID': '2024-09-28T07:00:00-IPS-001'}
1,2024-09-26T20:00:00-CME-001,2024-09-26T20:00Z,{'activityID': '2024-09-26T19:41:00-FLR-001'}
2,2024-09-29T06:00:00-CME-001,2024-09-29T06:00Z,{'activityID': '2024-09-29T05:26:00-FLR-001'}
3,2024-10-01T01:09:00-CME-001,2024-10-01T01:09Z,{'activityID': '2024-09-30T23:37:00-FLR-001'}
4,2024-10-01T23:09:00-CME-001,2024-10-01T23:09Z,{'activityID': '2024-10-01T21:58:00-FLR-001'}


In [184]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors

def extract_activityID_from_dict(input_dict):
    try:
       activityID = input_dict.get("activityID", None)
       return activityID
    except (ValueError, TypeError) as e:
        print(f"Error: {str(e)}")
        return None

        # Log the error or print it for debugging

# sample_dict = {'activityID': '2024-05-02T15:49:00-MPC-001'}
# activity_id = extract_activityID_from_dict(sample_dict)
# print(activity_id)

extract_activityID_from_dict(df_new.loc[0, "linkedEvents"])


'2024-09-28T07:00:00-IPS-001'

In [185]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:

df_new["GST_ActivityID"] = df_new["linkedEvents"].apply(extract_activityID_from_dict)
df_new.head()

,activityID,startTime,linkedEvents,GST_ActivityID
0,2024-09-23T20:48:00-CME-001,2024-09-23T20:48Z,{'activityID': '2024-09-28T07:00:00-IPS-001'},2024-09-28T07:00:00-IPS-001
1,2024-09-26T20:00:00-CME-001,2024-09-26T20:00Z,{'activityID': '2024-09-26T19:41:00-FLR-001'},2024-09-26T19:41:00-FLR-001
2,2024-09-29T06:00:00-CME-001,2024-09-29T06:00Z,{'activityID': '2024-09-29T05:26:00-FLR-001'},2024-09-29T05:26:00-FLR-001
3,2024-10-01T01:09:00-CME-001,2024-10-01T01:09Z,{'activityID': '2024-09-30T23:37:00-FLR-001'},2024-09-30T23:37:00-FLR-001
4,2024-10-01T23:09:00-CME-001,2024-10-01T23:09Z,{'activityID': '2024-10-01T21:58:00-FLR-001'},2024-10-01T21:58:00-FLR-001


In [186]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:

df_new.dropna(subset="GST_ActivityID", inplace= True)
df_new.head()

,activityID,startTime,linkedEvents,GST_ActivityID
0,2024-09-23T20:48:00-CME-001,2024-09-23T20:48Z,{'activityID': '2024-09-28T07:00:00-IPS-001'},2024-09-28T07:00:00-IPS-001
1,2024-09-26T20:00:00-CME-001,2024-09-26T20:00Z,{'activityID': '2024-09-26T19:41:00-FLR-001'},2024-09-26T19:41:00-FLR-001
2,2024-09-29T06:00:00-CME-001,2024-09-29T06:00Z,{'activityID': '2024-09-29T05:26:00-FLR-001'},2024-09-29T05:26:00-FLR-001
3,2024-10-01T01:09:00-CME-001,2024-10-01T01:09Z,{'activityID': '2024-09-30T23:37:00-FLR-001'},2024-09-30T23:37:00-FLR-001
4,2024-10-01T23:09:00-CME-001,2024-10-01T23:09Z,{'activityID': '2024-10-01T21:58:00-FLR-001'},2024-10-01T21:58:00-FLR-001


In [187]:
# print out the datatype of each column in this DataFrame:

df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      38 non-null     object
 1   startTime       38 non-null     object
 2   linkedEvents    38 non-null     object
 3   GST_ActivityID  38 non-null     object
dtypes: object(4)
memory usage: 1.3+ KB


In [188]:
# Convert the 'GST_ActivityID' column to string format 
df_new["GST_ActivityID"] = df_new["GST_ActivityID"].astype("string")

# Convert startTime to datetime format  
df_new["startTime"] = pd.to_datetime(df_new["startTime"])

# Rename startTime to startTime_CME and activityID to cmeID

df_new.rename(columns= {"startTime": "startTime_CME"}, inplace= True)
df_new.rename(columns= {"activityID": "cmeID"}, inplace= True)

# Drop linkedEvents

df_new.drop(columns= ["linkedEvents"], inplace = True)

# Verify that all steps were executed correctly

df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           38 non-null     object             
 1   startTime_CME   38 non-null     datetime64[ns, UTC]
 2   GST_ActivityID  38 non-null     string             
dtypes: datetime64[ns, UTC](1), object(1), string(1)
memory usage: 1.0+ KB


In [189]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  

df_new = df_new[df_new["GST_ActivityID"].str.contains("GST")]
df_new.head()

,cmeID,startTime_CME,GST_ActivityID
10,2024-10-03T12:48:00-CME-001,2024-10-03 12:48:00+00:00,2024-10-07T12:00:00-GST-001
29,2024-10-09T02:12:00-CME-001,2024-10-09 02:12:00+00:00,2024-10-10T15:00:00-GST-001


### GST Data

In [190]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_url=f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

In [191]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)

In [192]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

# Preview the first result in JSON format

# Use json.dumps with argument indent=4 to format data

print(json.dumps(gst_json[:1], indent=4))

[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "submissionTime": "2013-07-15T19:26Z",
        "versionId": 1
    }
]


In [193]:
# Convert gst_json to a Pandas DataFrame  

gst_df = pd.DataFrame(gst_json)

# Keep only the columns: activityID, startTime, linkedEvents

cols = ["gstID", "startTime", "linkedEvents"]

gst_df= gst_df[cols]

gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [194]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME

gst_df.dropna(subset = "linkedEvents", inplace= True)
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [195]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.

gst_df = gst_df.explode("linkedEvents", ignore_index= True).dropna()

In [196]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:

gst_df["CME_ActivityID"] = gst_df["linkedEvents"].apply(extract_activityID_from_dict)


# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:

gst_df.dropna(subset="CME_ActivityID", inplace= True)
gst_df.head()

,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [197]:
# Convert the 'gstID' column to string format 

gst_df["CME_ActivityID"] = gst_df["CME_ActivityID"].astype("string")

# Convert startTime to datetime format  

gst_df["startTime"] = pd.to_datetime(gst_df["startTime"])

# Rename startTime to startTime_GST 

gst_df.rename(columns= {"startTime": "startTime_GST"}, inplace= True)

# Drop linkedEvents

gst_df.drop(columns= ["linkedEvents"], inplace = True)

# Verify that all steps were executed correctly

gst_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    object             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  205 non-null    string             
dtypes: datetime64[ns, UTC](1), object(1), string(1)
memory usage: 4.9+ KB


In [198]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  

gst_df = gst_df[gst_df["CME_ActivityID"].str.contains("CME")]
gst_df.head()

,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [208]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(gst_df, df_new, how= "outer", left_on=["gstID", "CME_ActivityID"], right_on = ["GST_ActivityID","cmeID"])

merged_df.head(63)


# merged_df = pd.merge(
#     left=df_new, 
#     right=gst_df, 
#     how="outer",
#     left_on= ["gstID", "CME_ActivityID"], 
#     right_on= ["cmeID", "GST_ActivityID"], 

# )

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,NaN,NaT,<NA>
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,NaN,NaT,<NA>
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,NaN,NaT,<NA>
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,NaN,NaT,<NA>
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,NaN,NaT,<NA>
...,...,...,...,...,...,...
58,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001,NaN,NaT,<NA>
59,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001,NaN,NaT,<NA>
60,2024-04-19T18:00:00-GST-001,2024-04-19 18:00:00+00:00,2024-04-15T06:48:00-CME-001,NaN,NaT,<NA>
61,NaN,NaT,NaN,2024-10-03T12:48:00-CME-001,2024-10-03 12:48:00+00:00,2024-10-07T12:00:00-GST-001


In [209]:
# Verify that the new DataFrame has the same number of rows as cme and gst
merged_df.info

<bound method DataFrame.info of                           gstID             startTime_GST  \
0   2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
1   2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
2   2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
3   2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
4   2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   
..                          ...                       ...   
58  2024-03-24T12:00:00-GST-001 2024-03-24 12:00:00+00:00   
59  2024-03-24T12:00:00-GST-001 2024-03-24 12:00:00+00:00   
60  2024-04-19T18:00:00-GST-001 2024-04-19 18:00:00+00:00   
61                          NaN                       NaT   
62                          NaN                       NaT   

                 CME_ActivityID                        cmeID  \
0   2013-06-02T20:24:00-CME-001                          NaN   
1   2013-09-29T22:40:00-CME-001                          NaN   
2   2013-12-04T23:12:00-CME-001            

### Computing the time it takes for a CME to cause a GST

In [211]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df["timeDiff"] = merged_df["startTime_GST"] - merged_df["startTime_CME"]
merged_df

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID,timeDiff
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,NaN,NaT,<NA>,NaT
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,NaN,NaT,<NA>,NaT
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,NaN,NaT,<NA>,NaT
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,NaN,NaT,<NA>,NaT
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,NaN,NaT,<NA>,NaT
...,...,...,...,...,...,...,...
58,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001,NaN,NaT,<NA>,NaT
59,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001,NaN,NaT,<NA>,NaT
60,2024-04-19T18:00:00-GST-001,2024-04-19 18:00:00+00:00,2024-04-15T06:48:00-CME-001,NaN,NaT,<NA>,NaT
61,NaN,NaT,NaN,2024-10-03T12:48:00-CME-001,2024-10-03 12:48:00+00:00,2024-10-07T12:00:00-GST-001,NaT


In [213]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 

time_diff = merged_df["timeDiff"].describe()
time_diff


count      0
mean     NaT
std      NaT
min      NaT
25%      NaT
50%      NaT
75%      NaT
max      NaT
Name: timeDiff, dtype: object

### Exporting data in csv format

In [28]:
# Export data to CSV without the index
merged_df.to_csv("../Documents/data-sourcing-challenge/output_NASA.csv", index=False)